In [1]:
! pip uninstall flask
! pip install flask
! pip install bentoml
! pip install mlflow # mlflow
! pip install pyngrok # 로컬 개발환경을 원격으로 공유해주는 라이브러리

Found existing installation: Flask 1.1.4
Uninstalling Flask-1.1.4:
  Would remove:
    /usr/local/bin/flask
    /usr/local/lib/python3.7/dist-packages/Flask-1.1.4.dist-info/*
    /usr/local/lib/python3.7/dist-packages/flask/*
Proceed (y/n)? y
  Successfully uninstalled Flask-1.1.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 101 kB 9.1 MB/s 
     |████████████████████████████████| 96 kB 5.4 MB/s 
     |████████████████████████████████| 133 kB 65.2 MB/s 
     |████████████████████████████████| 232 kB 57.2 MB/s 
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.0.1
    Uninstalling MarkupSafe-2.0.1:
      Successfully uninstalled MarkupSafe-2.0.1
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 1.0.1
    Uninstalling Werkzeug-1.0.1:
      Successfully uninstalled Werkzeug-1.0.1
  Attempting uninstall: Jinja2
    Found existing installation

In [2]:
import torch.nn as nn
import torch
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
import numpy as np
import torch.optim as optim
import mlflow # mlflow 사용을 위해

# Model 정의

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784,100)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(100,100)
        self.fc3 = nn.Linear(100,10)
    def forward(self, x):
        x1 = self.fc1(x)
        x2 = self.relu(x1)
        x3 = self.fc2(x2)
        x4 = self.relu(x3)
        x5 = self.fc3(x4)

        return x5


# Dataset 정의  
## MNIST Dataset을 사용하여 학습,검증을 합니다.

In [4]:
download_root = 'MNIST_data/'

train_dataset = datasets.MNIST(root=download_root,
                         train=True,
                         transform = transforms.ToTensor(),
                         download=True)
                         
test_dataset = datasets.MNIST(root=download_root,
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)    

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



# Batch_size, Train, Test Dataloader 정의

In [5]:
batch_size = 100
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)


# 학습률, Optimizer 정의

In [6]:
model = Net()
model.zero_grad()
loss_function = nn.CrossEntropyLoss()
learning_rate = 0.01
optimizer = optim.SGD(model.parameters(), lr=learning_rate)


# Epoch 정의 (10번)


In [7]:
total_batch = len(train_loader)
epochs = 10
print(len(train_loader))

600


# MLflow, Ngrok 환경설정

In [8]:
experiment_name = 'mnist_practice' # 실험명, 실험관리를 용이하게 해줍니다. 


if not mlflow.get_experiment_by_name(experiment_name): 
  mlflow.create_experiment(name=experiment_name)
experiment = mlflow.get_experiment_by_name(experiment_name)

get_ipython().system_raw("mlflow ui --port 5000 &") # mlflow 포트 지정 및 백그라운드 실행을 위한 shell 코드.

from pyngrok import ngrok

ngrok.kill() # 혹시 열려있는 프로세스가 있으면 kill해줍니다.

NGROK_AUTH_TOKEN = "" # 개인 토큰입니다.https://dashboard.ngrok.com/get-started/your-authtoken 를 통해서 받아올 수 있습니다. 
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

ngrok_tunnel = ngrok.connect(addr="5000",proto="http",bind_tls=True) # tunnel을 설정해주는 코드입니다. mlflow의경우에는 5000번 포트를 사용합니다.
print("MLflow Tracking UI:", ngrok_tunnel.public_url) # 이 코드가 실행되고 나오는 url을 통해 MLflow에 접속할 수 있습니다.

MLflow Tracking UI: https://a0a7-35-199-26-52.ngrok.io


# Seed 고정

In [9]:
import torch.backends.cudnn as cudnn
import random
seed = 0
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(seed)

# Train, Track
## 학습을 진행하고 MLflow로 학습을 기록하는 부분입니다.

In [13]:
train_loss_list = []
train_acc_list = []

val_loss_list = []
val_acc_list = []

best_accuracy = 0
with mlflow.start_run(experiment_id=experiment.experiment_id):
  for epoch in range(epochs):
      cost=0
      model.train()
      train_accuracy = 0
      train_loss = 0
      for images, labels in train_loader:
          images = images.reshape(100,784)
          
          optimizer.zero_grad() # 변화도 매개변수 0
          
          #forward
          #pred = model.forward(images)
          #loss = loss_function(pred, labels)
          pred = model(images)
          loss = loss_function(pred,labels)
          prediction = torch.argmax(pred,1)
          correct = (prediction == labels)
          train_accuracy += correct.sum().item() / 60000
          train_loss += loss.item() / 600
          
          #backward
          loss.backward()
          
          #Update
          optimizer.step()
          
          cost += loss
      
      with torch.no_grad(): #미분하지 않겠다는 것
          total = 0
          correct=0
          for images, labels in test_loader:
              images = images.reshape(100,784)

              outputs = model(images)
              _,predict = torch.max(outputs.data, 1)

              total += labels.size(0)
              correct += (predict==labels).sum() # 예측한 값과 일치한 값의 합

      avg_cost = cost / total_batch
      accuracy = 100*correct/total
      
      val_loss_list.append(avg_cost.detach().numpy())
      val_acc_list.append(accuracy)

      if accuracy > best_accuracy:
        torch.save(model.state_dict(),'model.pt')
        best_accuracy = accuracy
        print(f"Save Model(Epoch: {epoch+1}, Accuracy: {best_accuracy:.5})")
      
      print("epoch : {} | loss : {:.6f}" .format(epoch+1, avg_cost))
      print("Accuracy : {:.2f}".format(100*correct/total))
      mlflow.log_param('learning-rate',learning_rate) # mlflow.log_param 을 사용하여 MLflow에 파라미터들을 기록할 수 있습니다.
      mlflow.log_param('epoch',epochs)
      mlflow.log_param('batch_size',batch_size)
      mlflow.log_param('seed',seed)
      mlflow.log_metric('train_accuracy',train_accuracy) # mlflow.log_metric을 사용하여 MLflow에 성능평가를 위한 metric을 기록할 수 있습니다.
      mlflow.log_metric('train_loss',train_loss)
      mlflow.log_metric('valid_accuracy',accuracy)
      mlflow.log_metric('valid_loss',avg_cost)
      mlflow.pytorch.log_model(model,'model') # pytorch.log_model 을 통해 모델을 저장할 수 있습니다.
      print("------")

2022/10/27 06:18:09 WARNING mlflow.utils.requirements_utils: Found torch version (1.12.1+cu113) contains a local version label (+cu113). MLflow logged a pip requirement for this package as 'torch==1.12.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


Save Model(Epoch: 1, Accuracy: 93.42)
epoch : 1 | loss : 0.236729
Accuracy : 93.42


2022/10/27 06:18:12 WARNING mlflow.utils.requirements_utils: Found torch version (1.12.1+cu113) contains a local version label (+cu113). MLflow logged a pip requirement for this package as 'torch==1.12.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


------


2022/10/27 06:18:20 WARNING mlflow.utils.requirements_utils: Found torch version (1.12.1+cu113) contains a local version label (+cu113). MLflow logged a pip requirement for this package as 'torch==1.12.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


...

------


2022/10/27 06:19:54 WARNING mlflow.utils.requirements_utils: Found torch version (1.12.1+cu113) contains a local version label (+cu113). MLflow logged a pip requirement for this package as 'torch==1.12.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


Save Model(Epoch: 10, Accuracy: 94.64)
epoch : 10 | loss : 0.175609
Accuracy : 94.64


2022/10/27 06:19:57 WARNING mlflow.utils.requirements_utils: Found torch version (1.12.1+cu113) contains a local version label (+cu113). MLflow logged a pip requirement for this package as 'torch==1.12.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


------


In [28]:
import json
import bentoml
import numpy as np
import pandas as pd
from torch import cuda

In [24]:
# mlflow 모델 저장
mlflow.pytorch.save_model(model, './model_mlflow')

# bentoml model import
bentoml.mlflow.import_model("torch_mmnist_model", model_uri='./model_mlflow')

MlflowException: ignored

In [29]:
with mlflow.start_run():
    mlflow.pytorch.log_model(model, artifact_path="pytorch-model")

    model_uri = mlflow.get_artifact_uri("pytorch-model")
    bento_model = bentoml.mlflow.import_model(
        'mlflow_pytorch_mnist',
        model_uri,
        signatures={'predict': {'batchable': True}}
    )

2022/10/27 06:39:39 WARNING mlflow.utils.requirements_utils: Found torch version (1.12.1+cu113) contains a local version label (+cu113). MLflow logged a pip requirement for this package as 'torch==1.12.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2022/10/27 06:39:42 WARNING mlflow.utils.requirements_utils: Found torch version (1.12.1+cu113) contains a local version label (+cu113). MLflow logged a pip requirement for this package as 'torch==1.12.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


In [30]:
!bentoml models list mlflow_pytorch_mnist

 Tag                            Module          Size        Creation Time       
 mlflow_pytorch_mnist:ennaslc…  bentoml.mlflow  355.65 KiB  2022-10-27 06:39:42 
 mlflow_pytorch_mnist:2tllvqs…  bentoml.mlflow  355.65 KiB  2022-10-27 06:23:11 


In [56]:
with open('./test_input_arr.json', 'r') as f:
  test_input_arr = np.array(json.load(f), dtype=np.float32)
input_df = pd.DataFrame(test_input_arr)

In [45]:
bento_model = bentoml.mlflow.get("mlflow_pytorch_mnist:latest")
mlflow_model_path = bento_model.path_of(bentoml.mlflow.MLFLOW_MODEL_FOLDER)
device = 'cuda' if cuda.is_available() else 'cpu'

loaded_pytorch_model = mlflow.pytorch.load_model(mlflow_model_path)
loaded_pytorch_model.to(device)
loaded_pytorch_model.eval()
with torch.no_grad():
    input_tensor = torch.from_numpy(test_input_arr).to(device)
    predictions = loaded_pytorch_model(input_tensor)

In [46]:
pyfunc_model: mlflow.pyfunc.PyFuncModel = bentoml.mlflow.load_model("mlflow_pytorch_mnist:latest")
predictions = pyfunc_model.predict(test_input_arr)

In [50]:
runner = bentoml.mlflow.get("mlflow_pytorch_mnist:latest").to_runner()
runner.init_local()
runner.predict.run(input_df)

,0,1,2,3,4,5,6,7,8,9
0,1.786066,-4.454893,4.527348,5.522169,-5.885814,0.564083,-10.757994,11.019074,-1.851699,2.301114
1,4.239511,2.233873,10.294967,5.826038,-11.726528,2.888166,3.500257,-7.664067,1.983901,-10.928630
2,-4.752887,6.412834,1.323531,0.703690,-1.632512,-0.522275,-0.122782,1.290266,0.377384,-2.278572


In [55]:
!bentoml serve service:svc --reload

2022-10-27T08:10:43+0000 [INFO] [cli] Prometheus metrics for HTTP BentoServer from "service:svc" can be accessed at http://localhost:3000/metrics.
2022-10-27T08:10:43+0000 [INFO] [cli] Starting development HTTP BentoServer from "service:svc" running on http://0.0.0.0:3000 (Press CTRL+C to quit)
2022-10-27 08:10:45 circus[905] [INFO] Loading the plugin...
2022-10-27 08:10:45 circus[905] [INFO] Endpoint: 'tcp://127.0.0.1:43327'
2022-10-27 08:10:45 circus[905] [INFO] Pub/sub: 'tcp://127.0.0.1:52017'
2022-10-27T08:10:45+0000 [INFO] [observer] Watching directories: ['/content', '/root/bentoml/models']
2022-10-27T08:11:47+0000 [ERROR] [cli] Exception in callback <bound method Arbiter.manage_watchers of <circus.arbiter.Arbiter object at 0x7fa7b5b42e90>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/circus/util.py", line 1130, in _run
    val = self.callback()
  File "/usr/local/lib/python3.7/dist-packages/circus/util.py", line 1043, in wrapper
    % arbiter